In [33]:
import torch 
import torch.utils.data as data
import torchvision
from skimage import io
import matplotlib.pyplot as plt
import prep_ecg
from torchvision.transforms import ToTensor

In [15]:
# data=m4p.loadmat('DREAMER.mat')['DREAMER']['Data']
# baseline=baseline_ds(data)
# stimuli=stimuli_ds(data)
# score=score_dict(data)
# for i in range(0,18):
#     select_n_slice(0,i,baseline,10,'cwt',True,score)

In [73]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [84]:
ecg_image=torchvision.datasets.ImageFolder(root='images/',transform=ToTensor())
train=torch.utils.data.DataLoader(ecg_image,batch_size=len(ecg_image),shuffle=True,num_workers=0)


In [85]:
%%time
print(len(ecg_image))
i=0
for i_batch, sample_batched in enumerate(train):
    print((sample_batched[0].size()))
    print(i_batch)


108
torch.Size([108, 3, 47, 137])
0
CPU times: user 54.2 ms, sys: 58.4 ms, total: 113 ms
Wall time: 123 ms


In [ ]:
class mydataset(data.Dataset):
    def __init__(self, images, labels):
        self.images=images
        self.labels=labels
    def __getitem__(self,index):
        